Open Play Shots - Expected Behind Model - Model Tuning and Building - GBM

In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import open_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Model Version

In [4]:
model_version = 1
model_name = 'expected_behind_open'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [5]:
RESPONSE = ModellingDataContract.RESPONSE_BEHIND

Load Data

In [6]:
df_modelling = pd.read_csv(open_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
9211,211,kickIn,behind,1705.0,4,744,775.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,56.0,-36.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,1,0,1,108.0,33.0,75.0,Kick,Gather,Kick,Handball Received,56.0,48.0,7.0,4.0,-36.0,-44.0,-47.0,-37.0,775.0,771.0,769.0,766.0,31.0,4.0,11.313708,24.0,36.0,43.266615,0.982794,56.309932,0.082503,4.727050,0.258900,14.833859,576.0,3.178054,False,56,True
9212,212,kickIn,goal,1740.0,4,787,900.0,Geelong,Geelong,Brandan Parfitt,Brandan_Parfitt,Kick,75.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,6,0,109.0,34.0,75.0,Kick,Handball Received,Handball,Loose Ball Get,75.0,75.0,76.0,75.0,0.0,-1.0,-5.0,-6.0,900.0,899.0,898.0,897.0,113.0,1.0,1.000000,5.0,0.0,5.000000,0.000000,0.000000,2.168022,124.218492,-1.429422,-81.899864,25.0,1.609438,False,76,True
9213,218,possGain,goal,1837.0,4,1136,1198.0,Geelong,Geelong,Sam De Koning,Sam_De_Koning,Kick,78.0,1.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,6,0,115.0,40.0,75.0,Kick,Gather,Kick,Handball Received,78.0,78.0,65.0,64.0,1.0,1.0,-12.0,-9.0,1198.0,1193.0,1190.0,1188.0,62.0,5.0,0.000000,2.0,1.0,2.236068,0.463648,26.565051,-2.442748,-139.959156,-0.440569,-25.242748,4.0,0.693147,False,96,False
9214,227,throwIn,goal,1901.0,4,1452,1458.0,Geelong,Geelong,Joel Selwood,Joel_Selwood,Kick,37.0,17.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,6,0,121.0,46.0,75.0,Kick,Gather,Knock On,Handball,37.0,37.0,43.0,34.0,17.0,19.0,20.0,25.0,1458.0,1457.0,1455.0,1453.0,6.0,1.0,2.000000,43.0,17.0,46.238512,0.376490,21.571307,0.129338,7.410516,0.403551,23.121747,1849.0,3.761200,False,98,False
9215,233,possGain,goal,1971.0,4,1744,1753.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,48.0,-20.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,0,6,133.0,46.0,87.0,Kick,Bounce,Handball Received,Handball,48.0,38.0,35.0,37.0,-20.0,-31.0,-33.0,-37.0,1753.0,1752.0,1748.0,1747.0,9.0,1.0,14.866069,32.0,20.0,37.735925,0.558599,32.005383,0.144862,8.299977,0.461317,26.431498,1024.0,3.465736,False,53,True


In [7]:
training_data = df_modelling[df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]
test_data = df_modelling[~df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]

In [8]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]

In [9]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
4,38,possGain,miss,392.0,1,1148,1153.0,Sydney,Sydney,Errol Gulden,Errol_Gulden,Kick,40.0,-11.0,clanger,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,0,0,18.0,9.0,9.0,Kick,Handball Received,Handball,Loose Ball Get,40.0,36.0,34.0,32.0,-11.0,-10.0,-9.0,-7.0,1153.0,1150.0,1149.0,1148.0,5.0,3.0,4.123106,38.0,11.0,39.560081,0.281772,16.144339,0.156423,8.962369,0.495369,28.382581,1444.0,3.637586,False,88,False
6,46,possGain,goal,476.0,1,1383,1389.0,Sydney,Sydney,Isaac Heeney,Isaac_Heeney,Kick,56.0,6.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,6,0,6,19.0,9.0,10.0,Kick,Handball Received,Handball,Handball Received,56.0,58.0,58.0,58.0,6.0,21.0,25.0,26.0,1389.0,1389.0,1388.0,1387.0,6.0,0.0,15.132746,22.0,6.0,22.803509,0.266252,15.255119,0.276208,15.825576,0.987285,56.567262,484.0,3.091042,False,96,False
15,123,centreBounce,goal,1086.0,3,128,143.0,Brisbane Lions,Brisbane Lions,Zac Bailey,Zac_Bailey,Kick,32.0,8.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,6,6,0,51.0,58.0,-7.0,Kick,Handball Received,Handball,Gather,32.0,22.0,26.0,26.0,8.0,18.0,20.0,20.0,143.0,140.0,139.0,139.0,15.0,3.0,14.142136,46.0,8.0,46.690470,0.172191,9.865807,0.135683,7.774075,0.423021,24.237313,2116.0,3.828641,False,94,False
17,152,possGain,goal,1351.0,3,1224,1238.0,Sydney,Sydney,Logan McDonald,Logan_McDonald,Kick,75.0,0.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,6,0,6,58.0,77.0,-19.0,Kick,Handball Received,Handball,Uncontested Mark,75.0,74.0,67.0,62.0,0.0,-1.0,-2.0,-2.0,1238.0,1238.0,1236.0,1235.0,14.0,0.0,1.414214,3.0,0.0,3.000000,0.000000,0.000000,-15.483871,-887.160457,-0.692641,-39.685388,9.0,1.098612,False,83,False
23,185,throwIn,miss,1611.0,4,279,315.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,37.0,21.0,clanger,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,0,0,65.0,107.0,-42.0,Kick,Loose Ball Get,Kick Inside 50 Result,Kick Into F50,37.0,37.0,41.0,-6.0,21.0,24.0,29.0,43.0,315.0,314.0,306.0,305.0,36.0,1.0,3.000000,41.0,21.0,46.065171,0.473356,27.121303,0.124257,7.119376,0.387814,22.220095,1681.0,3.713572,False,98,False


Preprocess Data

In [10]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [11]:
X_train_preproc = X_train[ModellingDataContract.modelling_feature_list]
X_test_preproc = X_test[ModellingDataContract.modelling_feature_list]

In [12]:
X_train_preproc.shape, X_test_preproc.shape

((7353, 2), (1863, 2))

In [13]:
X_train_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
0,56.080300,0.383818
1,47.010637,0.416065
2,36.878178,0.708626
3,46.486557,0.328553
5,59.682493,0.546167


In [14]:
X_test_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
4,39.560081,0.281772
6,22.803509,0.266252
15,46.690470,0.172191
17,3.000000,0.000000
23,46.065171,0.473356


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [15]:
monotone_cols = []
monotone_increasing = [1]*len(monotone_cols)
monotonicity_dict = dict(zip(monotone_cols, monotone_increasing))

In [16]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train)

In [17]:
xgb_tuner.training_data.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
0,56.080300,0.383818
1,47.010637,0.416065
2,36.878178,0.708626
3,46.486557,0.328553
5,59.682493,0.546167


In [18]:
xgb_tuner.tune_hyperparameters()

[I 2023-06-28 10:54:42,868] A new study created in memory with name: no-name-95551b85-c0ef-42e2-9639-b68bf5370e96
[I 2023-06-28 10:54:42,921] Trial 0 finished with value: 0.6906884357452466 and parameters: {'max_depth': 8, 'min_child_weight': 75, 'eta': 0.010974281361225881, 'gamma': 6.486671581637299, 'lambda': 7.6562541042853915, 'alpha': 0.00025821978489642614, 'subsample': 0.4225652784368409, 'colsample_bytree': 0.690243990592699}. Best is trial 0 with value: 0.6906884357452466.
[I 2023-06-28 10:54:42,940] Trial 1 finished with value: 0.6743279224720243 and parameters: {'max_depth': 7, 'min_child_weight': 76, 'eta': 0.9579535929479172, 'gamma': 0.3064562005198486, 'lambda': 0.07754827749101734, 'alpha': 0.00014575766138219447, 'subsample': 0.4023911077526234, 'colsample_bytree': 0.7895351484049371}. Best is trial 1 with value: 0.6743279224720243.
[I 2023-06-28 10:54:43,049] Trial 2 finished with value: 0.685079484117694 and parameters: {'max_depth': 14, 'min_child_weight': 33, 'eta

Number of finished trials:  100
Best trial:
  Value: 0.6559365602353107
  Params: 
    max_depth: 12
    min_child_weight: 47
    eta: 0.5382512368591548
    gamma: 0.006046319753062553
    lambda: 0.00010088336474765176
    alpha: 1.1028710292707637
    subsample: 0.8517036990980913
    colsample_bytree: 0.3698030534729669


In [19]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 12,
 'min_child_weight': 47,
 'eta': 0.5382512368591548,
 'gamma': 0.006046319753062553,
 'lambda': 0.00010088336474765176,
 'alpha': 1.1028710292707637,
 'subsample': 0.8517036990980913,
 'colsample_bytree': 0.3698030534729669}

Training Model - SuperXGBClassifier class for training and predictions

In [20]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = {}

In [21]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test, 
                               params = params)

In [22]:
super_xgb.fit()

[0]	validation_0-logloss:0.68067	validation_1-logloss:0.68393
[1]	validation_0-logloss:0.66331	validation_1-logloss:0.67011
[2]	validation_0-logloss:0.66046	validation_1-logloss:0.67107
[3]	validation_0-logloss:0.65492	validation_1-logloss:0.66777
[4]	validation_0-logloss:0.65357	validation_1-logloss:0.66596
[5]	validation_0-logloss:0.65170	validation_1-logloss:0.66656
[6]	validation_0-logloss:0.65114	validation_1-logloss:0.66753
[7]	validation_0-logloss:0.64978	validation_1-logloss:0.66936
[8]	validation_0-logloss:0.64901	validation_1-logloss:0.66971
[9]	validation_0-logloss:0.64850	validation_1-logloss:0.67000
[10]	validation_0-logloss:0.64778	validation_1-logloss:0.67018
[11]	validation_0-logloss:0.64725	validation_1-logloss:0.67022
[12]	validation_0-logloss:0.64637	validation_1-logloss:0.66939
[13]	validation_0-logloss:0.64555	validation_1-logloss:0.67032
[14]	validation_0-logloss:0.64499	validation_1-logloss:0.67000
[15]	validation_0-logloss:0.64459	validation_1-logloss:0.67132
[1

In [23]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.3698030534729669, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.006046319753062553, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.5382512368591548, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=47, missing=nan, monotone_constraints={},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [24]:
super_xgb.xgb_model.get_booster().feature_names

['Distance_to_Middle_Goal', 'Angle_to_Middle_Goal']

In [25]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [26]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]

Check Average Predictions

In [27]:
train_probas.mean(), training_data[RESPONSE].mean(), train_probas.mean(), test_data[RESPONSE].mean()

(0.46955684, 0.46919624643002855, 0.46955684, 0.4643048845947397)

Export model

In [28]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [29]:
train_info = training_data[['Match_ID', "Chain_Number", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Behind,Distance_to_Middle_Goal,Angle_to_Middle_Goal,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,1,0,56.080300,0.383818,1,0.643548
1,202101_BrisbaneLions_Sydney,6,0,47.010637,0.416065,0,0.453056
2,202101_BrisbaneLions_Sydney,10,0,36.878178,0.708626,0,0.498231
3,202101_BrisbaneLions_Sydney,21,0,46.486557,0.328553,0,0.492757
4,202101_BrisbaneLions_Sydney,43,1,59.682493,0.546167,1,0.597553


In [30]:
test_info = test_data[['Match_ID', "Chain_Number", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Behind,Distance_to_Middle_Goal,Angle_to_Middle_Goal,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,38,0,39.560081,0.281772,0,0.473769
1,202101_BrisbaneLions_Sydney,46,0,22.803509,0.266252,0,0.424009
2,202101_BrisbaneLions_Sydney,123,0,46.690470,0.172191,0,0.418797
3,202101_BrisbaneLions_Sydney,152,0,3.000000,0.000000,0,0.148013
4,202101_BrisbaneLions_Sydney,185,0,46.065171,0.473356,1,0.507592


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")